## Day4-01: Agent Observability 代理可观测性

5 天 AI Agents 强化课程的第四天第一部分，我们将学习如何为 Agent 添加可观测性。

你将学会：

- 什么是 Agent 可观测性以及为什么它很重要
- 如何使用 DEBUG 日志和 ADK Web UI 调试 Agent
- 可观测性的三大支柱：日志、追踪、指标
- 如何在生产环境中使用 LoggingPlugin
- 何时使用不同类型的日志记录

## 精简总结：Agent 可观测性

### 核心问题

- **挑战**：AI Agent 失败方式不可预测，难以调试
- **解决方案**：Agent 可观测性让你完整了解 Agent 的决策过程

```
用户: "查找量子计算论文"
Agent: "我无法帮助处理该请求。"
你: 😭 为什么？？是提示词问题？缺少工具？API 错误？

DEBUG 日志: LLM 请求显示 "Functions: []"（没有工具！）
你: 🎯 原来如此！缺少 google_search 工具 - 简单修复！
```

### 可观测性三大支柱

| 支柱 | 作用 | 告诉你什么 |
|------|------|-----------|
| 日志 (Logs) | 记录单个事件 | **发生了什么** |
| 追踪 (Traces) | 连接事件序列 | **为什么得到这个结果** |
| 指标 (Metrics) | 汇总统计数据 | **整体表现如何** |

### 调试方式选择

| 场景 | 推荐方式 |
|------|------|
| 开发调试 | `adk web --log_level DEBUG` |
| 生产环境通用可观测性 | `LoggingPlugin()` |
| 自定义需求 | 自定义 Callbacks 和 Plugins |

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

---
## Section 1: 设置环境

### 1.1 配置你的 Gemini API 密钥

In [ ]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key 配置完成。")
            break
else:
    print(f"请在以下路径创建 .env 文件: {env_file}")

### 1.2 设置日志和清理旧文件

配置日志用于调试会话。以下代码确保我们也能捕获其他日志级别，如 DEBUG。

In [ ]:
import logging
import os

# 清理之前的日志文件
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 已清理 {log_file}")

# 配置 DEBUG 日志级别
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ 日志配置完成")

---
## Section 2: 使用 ADK Web UI 动手调试

### 2.1 创建一个 "研究论文查找" Agent

**我们的目标：** 构建一个研究论文查找 Agent，帮助用户查找任何主题的学术论文。

但首先，让我们故意创建一个**有问题的版本**来练习调试！我们将使用 `adk create` CLI 命令创建一个新的 Agent 文件夹。

In [ ]:
!adk create research-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

### 2.2 Agent 定义

接下来，让我们创建 root agent：
- 配置为 `LlmAgent`，给它一个名称、模型和指令
- `root_agent` 获取用户提示并将搜索委托给 `google_search_agent`
- 然后，Agent 使用 `count_papers` 工具来计算返回的论文数量

**👉 注意 root agent 的指令和 `count_papers` 工具的参数！**

In [ ]:
%%writefile research-agent/agent.py

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # 最大重试次数
    exp_base=7,  # 延迟乘数
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # 遇到这些 HTTP 错误时重试
)

# ---- 故意传入错误的数据类型 - `str` 而不是 `List[str]` ----
def count_papers(papers: str):
    """
    此函数计算字符串列表中的论文数量。
    Args:
      papers: 字符串列表，每个字符串是一篇研究论文。
    Returns:
      列表中的论文数量。
    """
    return len(papers)


# Google 搜索 Agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="使用 Google 搜索查找信息",
    instruction="""使用 google_search 工具查找给定主题的信息。返回原始搜索结果。
    如果用户要求论文列表，则给他们你找到的研究论文列表，而不是摘要。""",
    tools=[google_search]
)


# Root Agent
root_agent = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""你的任务是查找研究论文并计数。

    你必须始终遵循以下步骤：
    1) 使用 'google_search_agent' 查找用户提供的主题的研究论文。
    2) 然后，将论文传递给 'count_papers' 工具来计算返回的论文数量。
    3) 返回研究论文列表和论文总数。
    """,
    tools=[AgentTool(agent=google_search_agent), count_papers]
)

### 2.3 运行 Agent

现在让我们使用 `adk web --log_level DEBUG` CLI 命令运行 Agent。

**📍 关键是 `--log_level DEBUG`** - 这会显示：

* **完整的 LLM 提示**：发送给语言模型的完整请求，包括系统指令、历史记录和工具
* 来自服务的详细 API 响应
* 内部状态转换和变量值

其他日志级别包括：INFO、ERROR 和 WARNING。

在终端中运行以下命令启动 ADK Web UI：

```bash
adk web --log_level DEBUG research-agent
```

然后在浏览器中打开 http://localhost:8000

In [ ]:
# 或者直接在 notebook 中运行（会持续运行直到手动停止）
# !adk web --log_level DEBUG research-agent

### 📝 2.4 在 ADK Web UI 中测试 Agent

#### **👉 操作：在 ADK Web UI 中**

1. 从左上角的下拉菜单中选择 "research-agent"
2. 在聊天界面中输入：`Find latest quantum computing papers`（查找最新的量子计算论文）
3. 发送消息并观察响应。Agent 应该返回研究论文列表及其数量。

看起来我们的 Agent 工作了，我们得到了响应！🤔 **但等等，论文数量是不是异常地大？让我们查看日志和追踪。**

#### **👉 操作：Events 标签页 - 详细追踪**

1. 在 Web UI 中，点击左侧边栏的 **"Events"** 标签页
2. 你会看到所有 Agent 操作的时间顺序列表
3. 点击任何事件在底部面板展开其详细信息
4. 尝试点击 **"Trace"** 按钮查看每个步骤的时间信息
5. **点击 `execute_tool count_papers` span。你会看到对 `count_papers` 的函数调用返回了那个大数字作为响应**
6. 让我们看看作为输入传递给这个函数的是什么
7. **找到对应 `count_papers` 函数调用的 `call_llm` span**

#### **👉 操作：在 Events 中检查函数调用**

- 点击特定的 span 打开 Events 标签页
- 检查 `function_call`，关注 `papers` 参数
- 注意 `root_agent` 将 `papers` 列表作为 **str** 而不是 **List[str]** 传递 - 这就是我们的 bug！

![Demo](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/observability-demo.gif)

### 2.5 你的回合 - 修复它！👾

将 `count_papers` 工具中 `papers` 参数的数据类型更新为 `List[str]`，然后重新运行 `adk web` 命令！

---

## ‼️ **停止 ADK Web UI** 🛑

**为了运行本 notebook 后续部分的代码，** 请停止你启动 `adk web` 的运行中的 cell。

否则，只要 ADK Web UI 在运行，那个运行中的 cell 就会阻止其他 cell 运行。

---

### 2.6 通过本地日志调试

你也可以选择检查本地 DEBUG 日志来找到根本原因。运行以下 cell 打印日志文件的内容。查找详细日志如：
```
DEBUG - google_adk.models.google_llm - LLM Request: ...
DEBUG - google_adk.models.google_llm - LLM Response: ...
```

In [ ]:
# 检查有问题的 agent 的 DEBUG 日志
print("🔍 检查 web 服务器日志以获取调试线索...\n")
!cat logger.log

**你现在可以从日志和 adk web 回答的其他可观测性问题：**
- **效率**：Agent 是否做出了最优的工具选择？
- **推理质量**：提示是否结构良好且上下文适当？
- **性能**：查看追踪以识别哪些步骤耗时最长？
- **故障诊断**：当出现问题时，它到底在哪里失败的？

**关键学习：** 核心调试模式：`症状 → 日志 → 根本原因 → 修复`。

**调试胜利：** 你刚刚从 "Agent 神秘地失败了" 变成了 "我确切知道为什么以及如何修复它！" 这就是可观测性的力量！

---
## Section 3: 生产环境中的日志记录

**🎯 太好了！你现在可以使用 ADK Web UI 和 DEBUG 日志调试 Agent 故障。**

但是当你超越开发阶段会发生什么？需要超越 Web UI 的现实场景：

**❌ 问题 1：生产环境部署**
```
你: "让我打开 ADK Web UI 检查 Agent 为什么失败"
运维: "嗯... 这是生产服务器。没有 Web UI 访问权限。"
你: 😱 "我怎么调试生产问题？"
```

**❌ 问题 2：自动化系统**
```
你: "Agent 每天在我们的管道中运行 1000 次"
老板: "哪些运行慢？我们的成功率是多少？"
你: 😰 "我得手动检查 Web UI 1000 次..."
```

**💡 解决方案：**

我们需要一种方法来捕获可观测性数据，换句话说，**在代码中添加日志**。

👉 在传统软件开发中，这是通过在 Python 函数中添加日志语句来完成的 - **Agent 也不例外！** 我们需要向 Agent 添加日志语句，常见的方法是在 **Plugins** 中添加日志语句。

### 3.1 如何为生产可观测性添加日志？

Plugin 是一个自定义代码模块，在 Agent 生命周期的各个阶段自动运行。Plugin 由 "**Callbacks**" 组成，提供中断 Agent 流程的钩子。可以这样理解：

- **你的 Agent 工作流**：用户消息 → Agent 思考 → 调用工具 → 返回响应
- **Plugin 接入这个流程**：Agent 开始前 → 工具运行后 → LLM 响应时 → 等等
- **Plugin 包含你的自定义代码**：日志记录、监控、安全检查、缓存等

![Plugins Callbacks](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/plugins-callbacks.png)

### Callbacks

Callbacks 是 **Plugin 内部的原子组件** - 这些只是在 Agent 生命周期特定点运行的 Python 函数！**Callbacks 组合在一起创建 Plugin。**

有不同类型的 callbacks：
* **before/after_agent_callbacks** - 在 Agent 调用前/后运行
* **before/after_tool_callbacks** - 在工具调用前/后运行
* **before/after_model_callbacks** - 类似地，在 LLM 模型调用前/后运行
* **on_model_error_callback** - 遇到模型错误时运行

![Types of Callbacks](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/types_of_callbacks.png)

### 3.2 Plugin 是什么样子的？

In [ ]:
print("----- 示例 PLUGIN - 什么都不做 ----- ")

import logging
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.plugins.base_plugin import BasePlugin


# 适用于所有 agent 和 model 调用
class CountInvocationPlugin(BasePlugin):
    """一个自定义 plugin，用于计算 agent 和工具的调用次数。"""

    def __init__(self) -> None:
        """使用计数器初始化 plugin。"""
        super().__init__(name="count_invocation")
        self.agent_count: int = 0
        self.tool_count: int = 0
        self.llm_request_count: int = 0

    # Callback 1: 在 agent 调用前运行。你可以在这里添加任何自定义逻辑。
    async def before_agent_callback(
        self, *, agent: BaseAgent, callback_context: CallbackContext
    ) -> None:
        """计算 agent 运行次数。"""
        self.agent_count += 1
        logging.info(f"[Plugin] Agent 运行次数: {self.agent_count}")

    # Callback 2: 在 model 调用前运行。你可以在这里添加任何自定义逻辑。
    async def before_model_callback(
        self, *, callback_context: CallbackContext, llm_request: LlmRequest
    ) -> None:
        """计算 LLM 请求次数。"""
        self.llm_request_count += 1
        logging.info(f"[Plugin] LLM 请求次数: {self.llm_request_count}")

**关键洞察**：你只需在 runner 上注册一次 plugin，它就会自动应用到系统中的**每个 agent、工具调用和 LLM 请求**，按照你的定义执行。在[这里](https://google.github.io/adk-docs/plugins/#plugin-callback-hooks)阅读更多关于 Plugin hooks 的内容。

你可以按照下图中的数字来理解流程。

![Count Invocation Plugin](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/count-invocation-plugin.png)

### 3.3 ADK 的内置 `LoggingPlugin`

但你不必定义所有的 callbacks 和 plugins 来捕获 ADK 中的*标准*可观测性数据。相反，ADK 提供了一个内置的 **LoggingPlugin**，自动捕获所有 agent 活动：

- 🚀 用户消息和 agent 响应
- ⏱️ 性能分析的时间数据
- 🧠 用于调试的 LLM 请求和响应
- 🔧 工具调用和结果
- ✅ 完整的执行追踪

#### Agent 定义

让我们使用与之前演示相同的 Agent - 研究论文查找器！

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # 最大重试次数
    exp_base=7,  # 延迟乘数
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # 遇到这些 HTTP 错误时重试
)


def count_papers(papers: List[str]):
    """
    此函数计算字符串列表中的论文数量。
    Args:
      papers: 字符串列表，每个字符串是一篇研究论文。
    Returns:
      列表中的论文数量。
    """
    return len(papers)


# Google 搜索 Agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="使用 Google 搜索查找信息",
    instruction="使用 google_search 工具查找给定主题的信息。返回原始搜索结果。",
    tools=[google_search],
)

# Root Agent
research_agent_with_plugin = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""你的任务是查找研究论文并计数。
   
   你必须遵循以下步骤：
   1) 使用 'google_search_agent' 查找用户提供的主题的研究论文。
   2) 然后，将论文传递给 'count_papers' 工具来计算返回的论文数量。
   3) 返回研究论文列表和论文总数。
   """,
    tools=[AgentTool(agent=google_search_agent), count_papers],
)

print("✅ Agent 创建完成")

### 3.4 将 LoggingPlugin 添加到 Runner

以下代码创建 `InMemoryRunner`。这用于以编程方式调用 agent。

**要在上述研究 agent 中使用 `LoggingPlugin`：**
1) 导入 plugin
2) 在初始化 `InMemoryRunner` 时添加它

In [ ]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. 导入 Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=research_agent_with_plugin,
    plugins=[
        LoggingPlugin()
    ],  # <---- 2. 添加 plugin。处理所有 Agent 的标准可观测性日志记录
)

print("✅ Runner 配置完成")

让我们现在使用 `run_debug` 函数运行 agent。

In [ ]:
print("🚀 使用 LoggingPlugin 运行 agent...")
print("📊 观察下面的综合日志输出：\n")

response = await runner.run_debug("查找关于量子计算的最新论文")

---

## 📊 总结

**❓ 什么时候使用哪种类型的日志记录？**
1. **开发调试？** → 使用 `adk web --log_level DEBUG`
2. **常见生产可观测性？** → 使用 `LoggingPlugin()`
3. **自定义需求？** → 构建自定义 Callbacks 和 Plugins

### 试试看！

👉 通过实现一个**自定义 ADK plugin** 来扩展 agent 的可观测性，该 plugin 跟踪并报告会话期间进行的工具调用总数。

## 🎯 恭喜！

**你现在知道如何：**

- ✅ 通过 DEBUG 日志和 ADK Web UI 调试 agent 故障
- ✅ 使用核心调试模式：症状 → 日志 → 根本原因 → 修复
- ✅ 使用 `LoggingPlugin` 为生产系统扩展可观测性
- ✅ 了解何时使用不同的日志类型

### 📚 资源

**参考 ADK 文档了解更多关于可观测性的内容：**

- [ADK 可观测性文档](https://google.github.io/adk-docs/observability/logging/) - ADK 日志记录完整指南
- [自定义 Plugin](https://google.github.io/adk-docs/plugins/) - 构建你自己的 Plugins
- [外部集成](https://google.github.io/adk-docs/observability/cloud-trace/) - 探索与 ADK 的第三方可观测性集成

### 🎯 下一步

准备好迎接下一个挑战了吗？继续下一个 notebook 学习如何**评估 Agent**，确保它在生产环境中按预期工作。